# Clase Práctica #4 (Compilación)

En esta clase estaremos implementando un mecanismo genérico de **evaluación** de cadenas a partir de la especificación de **atributos** y **reglas** en la gramática. Diseñaremos concretamente las reglas de evaluación para la gramática del subconjunto de `HULK` con que hemos trabajado desde clases anteriores.

## Gramáticas Atributadas

Recordemos que una **gramática atributada** es una tupla $<G,A,R>$, donde:

* $G = <S,P,N,T>$ es una gramática libre del contexto,
* $A$ es un conjunto de atributos de la forma $X \cdot a$
  donde $X \in N \cup T$ y $a$ es un identificador único entre todos los atributos del mismo símbolo, y
* $R$ es un conjunto de reglas de la forma $<p_i, r_i>$ donde $p_i \in P$ es una producción $X \to Y_1, \ldots, Y_n$, y $r_i$ es una regla de la forma:
    1. $X \cdot a = f(Y_1 \cdot a_1, \ldots, Y_n \cdot a_n)$, o
    2. $Y_i \cdot a = f(X \cdot a_0, Y_1 \cdot a_1, \ldots, Y_n \cdot a_n)$.

Los atributos se dividen en dos conjuntos disjuntos: _atributos heredados_ y _atributos sintetizados_. En el caso (1) decimos que `a` es un _atributo sintetizado_, y en el caso (2), un _atributo heredado_.

Según esta distinción, estudiamos en conferencia condiciones suficientes para que una gramática fuera evaluable:

- Una gramática atributada es **s-atributada** si y solo si, para toda regla $r_i$ asociada a una producción $X \to Y_1, \ldots, Y_n$, se cumple que $r_i$ es de la forma $X \cdot a = f(Y_1 \cdot a_1, \ldots, Y_n \cdot a_n)$.

- Una gramática atributada es **l-atributada** si y solo si toda regla $r_i$ asociada a una producción $X \to Y_1, \ldots, Y_n$ es de una de las siguientes formas:
    - $X \cdot a = f(Y_1 \cdot a_1, \ldots, Y_n \cdot a_n)$, ó
    - $Y_i \cdot a_i = f(X \cdot a, Y_1 \cdot a_1, \ldots, Y_{i-1} \cdot a_{i-1})$.

### Especificación en _Python_

Continuaremos trabajando con la _API_ para gramáticas presentada en la clase anterior. Esta vez, la definición de símbolo, oración, producción, gramática, etc. se encuentra en el módulo `cmp` que se distribuye junto a este _notebook_.

Procedamos a importar las clases y métodos que nos interesan.

In [ ]:
from cmp.pycompiler import Symbol, NonTerminal, Terminal, EOF, Sentence, Epsilon, Production, Grammar
from cmp.utils import pprint, inspect

A la _API_ de gramáticas se añade una nueva clase: `AttributeProduction`. Con esta clase modelaremos las producciones de las gramáticas atributadas. Cada una de estas producciones se compone por:
- Un no terminal como cabecera. Accesible a través del campo `Left`.
- Una oración como cuerpo. Accesible a través del campo `Right`.
- Un conjunto de reglas para evaluar los atributos. Accesible a través del campo `atributes`.

Las producciones no deben ser instanciadas directamente con la aplicación de su constructor. En su lugar, se presentan las siguientes facilidades para formar producciones a partir de una instancia `G` de `Grammar` y un grupo de terminales y no terminales:
- Para definir una producción de la forma $B_0 \to B_1 B_2 ... B_n$ que:
    - Asocia a $B_0$ una regla $\lambda_0$ para sintetizar sus atributos, y
    - Asocia a $B_1 \dots B_n$ las reglas $\lambda_1 \dots \lambda_n$ que hereden sus atributos respectivamentes.
    
    ```python
    B0 %= B1 + B2 + ... + Bn, lambda0, lambda1, lambda2, ..., lambdaN
    ```
    
> Donde `lambda0`, `lambda1`, ..., `lambdaN` son funciones que reciben 2 parámetros.
> 1. Como primer parámetro los atributos heredados que se han computado para cada instancia de símbolo en la producción, durante la aplicación de esa instancia de producción específicamente. Los valores se acceden desde una lista de `n + 1` elementos. Los valores se ordenan según aparecen los símbolos en la producción, comenzando por la cabecera. Nos referiremos a esta colección como `inherited`.
> 2. Como segundo parámetro los atributos sintetizados que se han computado para cada instancia de símbolo en la producción, durante la aplicación de esa instancia de producción específicamente. Sigue la misma estructura que el primer parámetro. Nos referiremos a esta colección como `synteticed`.
>
> La función `lambda0` sintetiza los atributos de la cabecera. La evaluación de dicha función produce el valor de `synteticed[0]`. El resto de los atributos sintetizados de los símbolos de la producción se calcula de la siguiente forma:
> - En caso de que el símbolo sea un terminal, evalúa como su lexema.
> - En caso de que el símbolo sea un no terminal, se obtiene de evaluar la función `lambda0` en la instancia de producción correspondiente.
>
> La función `lambda_i`, con `i` entre 1 y `n`, computa los atributos heredados de la i-ésima ocurrencia de símbolo en la producción. La evaluación de dicha función produce el valor de `inherited[i]`. El valor de `inherited[0]` se obtiene como el atributo que heredó la instancia concreta del símbolo en la cabecera antes de comenzar a aplicar la producción.

- En caso de que no se vaya a sociar una regla a un símbolo se incluirá un `None`.
    ```python
       E %= T + X   ,  lambda h,s: s[2]  ,    None    ,   lambda h,s: s[1]
    # ___________     ________________     ________      ________________
    # producción  |    regla para E    |  sin regla  |     regla para X 
    ```
    > `[0]:` **`lambda h,s: s[2]`** al ser `lambda0` sintetiza el valor de `E`. Lo hace en función del valor que sintetiza `X` (accesible desde `s[2]`).  
    > `[1]:` **`None`** al ser `lambda1` indica que no se incluye regla para heredar valor a `T`.  
    > `[2]:` **`lambda h,s: s[1]`** al ser `lambda2` hereda un valor a `X`. Lo hace en función del valor que sintetiza `T` (accesible desde `s[1]`).

- No se deben definir múltiples producciones de la misma cabecera en una única sentencia.

In [ ]:
class AttributeProduction(Production):

    def __init__(self, nonTerminal, sentence, attributes):
        if not isinstance(sentence, Sentence) and isinstance(sentence, Symbol):
            sentence = Sentence(sentence)
        super(AttributeProduction, self).__init__(nonTerminal, sentence)

        self.attributes = attributes

    def __str__(self):
        return '%s := %s' % (self.Left, self.Right)

    def __repr__(self):
        return '%s -> %s' % (self.Left, self.Right)

    def __iter__(self):
        yield self.Left
        yield self.Right


    @property
    def IsEpsilon(self):
        return self.Right.IsEpsilon

#### Gramática de HULK

Completemos entonces la siguiente especificación de la gramática para `HULK` añadiendo las reglas necesarias.

`E` $\rightarrow$ `T X`  
`X` $\rightarrow$ `+ T X | - T X | epsilon`  
`T` $\rightarrow$ `F Y`  
`Y` $\rightarrow$ `* F Y | / F Y | epsilon`  
`F` $\rightarrow$ `( E ) | num`

In [ ]:
G = Grammar()
E = G.NonTerminal('E', True)
T, F, X, Y = G.NonTerminals('T F X Y')
plus, minus, star, div, opar, cpar, num = G.Terminals('+ - * / ( ) num')

############################ BEGIN PRODUCTIONS ############################
# ======================================================================= #
#                                                                         #
# ========================== { E --> T X } ============================== #
#                                                                         #
# E %= T + X, lambda h,s: s[2], None, lambda h,s: s[1]                    #
#                                                                         #
# =================== { X --> + T X | - T X | epsilon } ================= #
#                                                                         #
# X %= plus + T + X, None, None, None, None                               #
# X %= minus + T + X, None, None, None, None                              #
# X %= G.Epsilon, None                                                    #
#                                                                         #
# ============================ { T --> F Y } ============================ #
#                                                                         #
# T %= F + Y, None, None, None                                            #
#                                                                         #
# ==================== { Y --> * F Y | / F Y | epsilon } ================ #
#                                                                         #
# Y %= star + F + Y, None, None, None, None                               #
# Y %= div + F + Y, None, None, None, None                                #
# Y %= G.Epsilon, None                                                    #
#                                                                         #
# ======================= { F --> num | ( E ) } ========================= #
# F %= num, None, None                                                    #
# F %= opar + E + cpar, None, None, None, None                            #
#                                                                         #
# ======================================================================= #
############################# END PRODUCTIONS #############################

print(G)

## Parsing

En la clase pasada implementamos los algoritmos para calcular los conjuntos `first` y `follow`. Esta vez utilizaremos dichos conjuntos ya precomputados para nuestro subconjunto de `HULK`. Pasemos a importarlos desde el módulo `utils`.

In [ ]:
from cmp.languages import BasicHulk

hulk = BasicHulk(G)
firsts, follows = hulk.firsts, hulk.follows

De forma similar procederemos con los métodos `build_parsing_table` y `metodo_predictivo_no_recursivo` que devuelven la tabla _LL(1)_ y el parser _LL(1)_ respectivamente. Pasemos a importarlos desde el módulo `tools`.

In [ ]:
from cmp.tools.parsing import build_parsing_table
from cmp.tools.parsing import deprecated_metodo_predictivo_no_recursivo as metodo_predictivo_no_recursivo

# Testing table
M = build_parsing_table(G, firsts, follows)
assert M == hulk.table

# Testing parser
parser = metodo_predictivo_no_recursivo(G, M)
left_parse = parser([num, star, num, star, num, plus, num, star, num, plus, num, plus, num, G.EOF])
assert left_parse == [ 
   Production(E, Sentence(T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, Sentence(star, F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, Sentence(plus, T, X)),
   Production(T, Sentence(F, Y)),
   Production(F, Sentence(num)),
   Production(Y, G.Epsilon),
   Production(X, G.Epsilon),
]

## Evaluación

En la clase anterior asumimos que la cadena de entrada que queremos parsear es una lista de símbolos terminales. Aún así, notemos que en realidad la entrada no está compuesta solamente por estos símbolos. El parser trabaja con una secuencia de _tokens_, que como ya sabemos se componen de un _lexema_ y un _tipo_. Los símbolos terminales son justamente los tipos de los tokens y, por tanto, son los valores relevantes al parsear. Sin embargo, nuestro problema no termina al parsear sino que debemos ser capaces de evaluar, en el lenguaje actual, la expresión de `HULK` que se dió como entrada. Para ello, el lexema de los tokens juega un papel esencial ya que son estos los que capturan las particularidades de los valores de entrada. Por ejemplo, en el caso de `HULK`, para saber qué dos números se están operando es necesario considerar los lexemas.

A continuación se implementa la clase `Token` usada para modelar los tokens. Se puede acceder al lexema y tipo de cada token a través de los campos `lex` y `token_type` respectivamente.

In [ ]:
class Token:
    """
    Basic token class. 
    
    Parameters
    ----------
    lex : str
        Token's lexeme.
    token_type : Enum
        Token's type.
    """
    
    def __init__(self, lex, token_type):
        self.lex = lex
        self.token_type = token_type
    
    def __str__(self):
        return f'{self.token_type}: {self.lex}'
    
    def __repr__(self):
        return str(self)

Modifiquemos el generador de parsers para que acceda el tipo de token a través de la propiedad `token_type`.

In [ ]:
deprecated_metodo_predictivo_no_recursivo = metodo_predictivo_no_recursivo

**OJO: No ejecute la celda anterior ($\uparrow$) una vez ejecutadas las celdas que siguen a continuación ($\downarrow$)**

Redefiniremos la implementación del generador de parsers hacia una que *decore* la salida del actual. Esta nueva implementación simplemente extraerá de los tokens de entrada los respectivos tipos (`token_type`), y procederá de la misma forma que ya estaba implementada. Claramente, los hacemos de esta forma para reutilizar la versión que ya teníamos implementada, pero podríamos reescribir la implementación original para que al acceder al símbolo puntado por el cabezal (`a = w[cursor]`) accediera a su tipo a través del campo `token_type`.

In [ ]:
def metodo_predictivo_no_recursivo(G, M):
    parser = deprecated_metodo_predictivo_no_recursivo(G, M)
    def updated(tokens):
        return parser([t.token_type for t in tokens])
    return updated

Rápidamente podemos comprobar la efectividad del cambio:

In [ ]:
text = '5.9 + 4'
tokens = [ Token('5.9', num), Token('+', plus), Token('4', num), Token('$', G.EOF) ]
parser = metodo_predictivo_no_recursivo(G, M)
left_parse = parser(tokens)
left_parse

Pasemos finalmente a implementar un algoritmo de evaluación de la secuencia de tokens a partir del parse izquierdo.

In [ ]:
def evaluate_parse(left_parse, tokens):
    if not left_parse or not tokens:
        return
    
    left_parse = iter(left_parse)
    tokens = iter(tokens)
    result = evaluate(next(left_parse), left_parse, tokens)
    
    assert isinstance(next(tokens).token_type, EOF)
    return result
    

def evaluate(production, left_parse, tokens, inherited_value=None):
    head, body = production
    attributes = production.attributes
    
    # Insert your code here ...
    # > synteticed = ...
    # > inherited = ...
    # Anything to do with inherited_value?

    for i, symbol in enumerate(body, 1):
        if symbol.IsTerminal:
            assert inherited[i] is None
            # Insert your code here ...
        else:
            next_production = next(left_parse)
            assert symbol == next_production.Left
            # Insert your code here ...
    
    # Insert your code here ...
    # > return ...
    return


Y enseguida podemos comprobar la correctitud del algoritmo:

In [ ]:
result = evaluate_parse(left_parse, tokens)
print(f'{text} = {result}')
assert result == 9.9

## Completando el pipeline

Implementemos nuevamente un tokenizer muy básico. Asumiremos como de costumbre que las unidades léxicas relevantes están separadas por espacio (o sea, que los números y operadores están separados por al menos un espacio).

In [ ]:
fixed_tokens = {
    '+'  :   Token( '+', plus  ),
    '-'  :   Token( '-', minus ),
    '*'  :   Token( '*', star  ),
    '/'  :   Token( '/', div   ),
    '('  :   Token( '(', opar  ),
    ')'  :   Token( ')', cpar  ),
}

def tokenize_text(text):
    tokens = []

    for item in text.split():
        try:
            float(item)
            token = Token(item, num)
        except ValueError:
            try:
                token = fixed_tokens[item]
            except:
                raise Exception('Undefined token')
        tokens.append(token)

    eof = Token('$', G.EOF)
    tokens.append(eof)

    return tokens

Probemos algunas cadenas. Se realizará la siguiente cadena de transformaciones:
```
Entrada -> Tokens -> Parse Izquierdo -> Resultado
```    

In [ ]:
text = '1 - 1 - 1'
tokens = tokenize_text(text)
pprint(tokens, '================Tokens================')
left_parse = parser(tokens)
pprint(left_parse, '==============Left-Parse==============')
result = evaluate_parse(left_parse, tokens)
pprint(f'{text} = {result}', '================Result================')
assert result == -1

In [ ]:
text = '1 - ( 1 - 1 )'
tokens = tokenize_text(text)
pprint(tokens, '================Tokens================')
left_parse = parser(tokens)
pprint(left_parse, '==============Left-Parse==============')
result = evaluate_parse(left_parse, tokens)
pprint(f'{text} = {result}', '================Result================')
assert result == 1

## Propuestas

- Con el objetivo de simplificar la implementación de los algoritmos en la clase, la evaluación de los atributos se realizó posteriormente a que se obtuviera completamente el parse izquierdo. Sin embargo, vimos en conferencia que la evaluación de los atributos puede realizarse junto al proceso de parsing LL(1) si la gramática es _L-atributada_. Realice las modificaciones pertinentes para evaluar los atributos a medida que se parsea la cadena.